In [1]:
import boto3
import pandas as pd

In [2]:
# train_file = pd.read_csv('Downloads/twinkeydow-master/als_thing/data/train.csv', index_col=None, sep=',') \
#     .drop_duplicates()
train_file = pd.read_csv('Downloads/16kUser.csv', index_col=None, sep=',') \
    .drop_duplicates()

# train_file.columns = ['uid', 'repoid', 'uname', 'reponame', 'date']
train_file.columns = ['uid', 'repoid']
train_file = train_file.drop_duplicates()
train_file.dropna(inplace=True)
train_file.repoid = train_file.repoid.astype(int)
train_file.head()

,uid,repoid
0,8740207,3964514
1,8740207,85077558
2,8740207,101296881
3,8740207,60493101
4,4782875,110748052


In [3]:
# user_file = pd.read_csv('Downloads/twinkeydow-master/als_thing/data/user.csv', index_col=None, sep=',') \
#     .drop_duplicates()
user_file = pd.read_csv('Downloads/registerd_user_star_repo.csv', index_col=None, sep=',') \
    .drop_duplicates()

#the users that have alreday register out web app.
    
# user_file.columns = ['uid', 'repoid', 'uname', 'reponame', 'date']
user_file.columns = ['uid', 'repoid']
user_file = user_file.drop_duplicates()
user_file.dropna(inplace=True)
user_file.repoid = train_file.repoid.astype(int)
user_file.head()

,uid,repoid
0,13039951,3964514
1,13039951,85077558
2,5056560,101296881
3,5056560,60493101
4,5056560,110748052


In [4]:

# take only uid / repoid
train_file = train_file[['uid', 'repoid']]
train_file.head()

,uid,repoid
0,8740207,3964514
1,8740207,85077558
2,8740207,101296881
3,8740207,60493101
4,4782875,110748052


In [5]:
# same thing for user file
user_file = user_file[['uid', 'repoid']]
user_file.head()

,uid,repoid
0,13039951,3964514
1,13039951,85077558
2,5056560,101296881
3,5056560,60493101
4,5056560,110748052


In [6]:
import findspark
findspark.init("/usr/local/spark-2.3.0")

from pyspark import SparkContext
from pyspark.sql import SQLContext

from pyspark.mllib.recommendation import ALS

sc = SparkContext(appName="RR")
sqlContext = SQLContext(sc)

In [7]:
def first_two_column(row):
    return (int(row[0]), int(row[1]), 1.0)

In [8]:
training_rdd = sqlContext.createDataFrame(train_file).rdd
training_rdd = training_rdd \
    .map(first_two_column)

In [9]:
training_rdd.take(5)

[(8740207, 3964514, 1.0),
 (8740207, 85077558, 1.0),
 (8740207, 101296881, 1.0),
 (8740207, 60493101, 1.0),
 (4782875, 110748052, 1.0)]

In [10]:
# this takes a short while too
model = ALS.trainImplicit( \
    training_rdd,
    rank=16,
    iterations=10,
    lambda_=0.1,
    alpha=80.0
)

In [14]:
user_rdd = sqlContext.createDataFrame(user_file).rdd
user_rdd.take(5)

[Row(uid=13039951, repoid=3964514),
 Row(uid=13039951, repoid=85077558),
 Row(uid=5056560, repoid=101296881),
 Row(uid=5056560, repoid=60493101),
 Row(uid=5056560, repoid=110748052)]

In [15]:
# append user to training set and train again
training_rdd = training_rdd.union(user_rdd.map(first_two_column))
# this takes a short while too
model = ALS.trainImplicit( \
    training_rdd,
    rank=16,
    iterations=10,
    lambda_=0.1,
    alpha=80.0
)

In [16]:
user_id_file = user_file[['uid']].drop_duplicates()
user_id_file.head()




,uid
0,13039951
2,5056560
6,33183203
16,5561833
43,3495948


In [17]:
user_id_rdd = sqlContext.createDataFrame(user_id_file).rdd
user_id_rdd.take(10)

[Row(uid=13039951),
 Row(uid=5056560),
 Row(uid=33183203),
 Row(uid=5561833),
 Row(uid=3495948)]

In [28]:
# prediction input is (to_be_predict_uid, all_repo_id)
#user_id = user_rdd.take(1)[0].uid

# put the predicting into the loop
# need to use multi-thread? 
num = user_id_rdd.count()
for i in range(num):
    print(user_id_rdd.take(num)[i].uid)
user_id = user_rdd.take(5)[2].uid





13039951
5056560
33183203
5561833
3495948


In [ ]:
predict_input_rdd = training_rdd \
    .map(lambda row: (user_id, row[1])) \
    .distinct()
predict_input_rdd.take(20)

In [49]:
temp = predict_input_rdd.map(lambda row: (row[0])) \
    .distinct()
temp.take(5)

[13039951]

In [31]:
predictions = model.predictAll(predict_input_rdd).sortBy(lambda row: -row.rating)

In [32]:

# remove repos user already starred
# this is top 10 recommendations for user
# which user?
predictions.map(lambda row: (row.product, row)) \
    .subtractByKey(user_rdd.map(lambda row: (row[1], 0))) \
    .map(lambda row: row[0]) \
    .take(10)

[120538304,
 1039520,
 33614304,
 21872392,
 31792824,
 24953448,
 22790488,
 12256376,
 724712,
 3544424]